In [1]:
import numpy as np
import pandas as pd
cols = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship'
        ,'race','sex','capital-gain','capital-loss','hours-per-week','native-country','Salary']
adult_data = pd.read_csv('https://raw.githubusercontent.com/zekelabs/data-science-complete-tutorial/master/Data/adult.data.txt', names=cols)

In [2]:
adult_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    32561 non-null object
Salary            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [3]:
adult_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [8]:
#first remove ? in dataframe and replace with np.nan
cat_cols_df=adult_data.select_dtypes('object')

In [11]:
for col in cat_cols_df.columns:
    adult_data[col]=adult_data[col].str.strip().replace('?',np.nan)

In [13]:
adult_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         30725 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        30718 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    31978 non-null object
Salary            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [17]:
cat_cols=list(cat_cols_df.columns)
cat_cols.remove('Salary')

In [20]:
num_cols=list(adult_data.select_dtypes('int64').columns)

In [22]:
cat_cols,num_cols

(['workclass',
  'education',
  'marital-status',
  'occupation',
  'relationship',
  'race',
  'sex',
  'native-country'],
 ['age',
  'fnlwgt',
  'education-num',
  'capital-gain',
  'capital-loss',
  'hours-per-week'])

In [31]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [27]:
pipeline_num=Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
     ('scaler',MinMaxScaler())
])

In [35]:
pipeline_cat=Pipeline([
    ('imputer',SimpleImputer(strategy='constant',fill_value='missing')),
    ('encoding',OneHotEncoder(handle_unknown='ignore'))
])

In [36]:
preprocessor=ColumnTransformer([
    ('num',pipeline_num,num_cols),
    ('cat',pipeline_cat,cat_cols)

])

In [37]:
pipeline=Pipeline([
    ('preprocessor',preprocessor),
    ('classifier',MultinomialNB())
])

In [38]:
trainX,testX,trainY,testY=train_test_split(adult_data,adult_data.Salary)

In [40]:
pipeline.fit(trainX,trainY)
pipeline.score(testX,testY)

0.7986733816484461

In [43]:
testX[:1].to_dict()

{'age': {21753: 22},
 'workclass': {21753: 'Private'},
 'fnlwgt': {21753: 246965},
 'education': {21753: 'Some-college'},
 'education-num': {21753: 10},
 'marital-status': {21753: 'Never-married'},
 'occupation': {21753: 'Prof-specialty'},
 'relationship': {21753: 'Not-in-family'},
 'race': {21753: 'White'},
 'sex': {21753: 'Male'},
 'capital-gain': {21753: 0},
 'capital-loss': {21753: 0},
 'hours-per-week': {21753: 12},
 'native-country': {21753: 'United-States'},
 'Salary': {21753: '<=50K'}}

In [44]:
pipeline.predict_proba(testX[:1])

array([[0.96622302, 0.03377698]])

In [45]:
pipeline.classes_

array(['<=50K', '>50K'], dtype='<U5')